# RAS Methodology Deep Dive

This notebook explores the **Rademacher Anti-Serum (RAS)** methodology in detail, explaining:
1. Why traditional backtesting can be misleading
2. What RAS measures and how it works
3. The components of the RAS adjustment
4. How to interpret RAS results
5. Practical implications for strategy evaluation

## The Problem: Data Snooping Bias

When backtesting trading strategies, we face several sources of bias:

1. **Estimation Error**: Limited historical data gives noisy estimates
2. **Data Snooping**: Testing multiple strategies inflates the chance of finding one that "worked" by luck
3. **Overfitting**: Complex strategies may fit noise rather than signal

The **Sharpe ratio** is a common performance measure:

$$\text{Sharpe Ratio} = \frac{\text{Mean Return}}{\text{Standard Deviation}}$$

But the *empirical* Sharpe ratio from a backtest is an **overestimate** of the true Sharpe ratio we can expect going forward.

RAS provides a **statistically rigorous lower bound** on the true Sharpe ratio.

## Setup

In [ ]:
import numpy as np
import pandas as pd
import rademacher_backtest as rbt
import matplotlib.pyplot as plt

# Load data
prices = pd.read_csv('data/sample_prices.csv', index_col='date', parse_dates=True)
loader = rbt.DataFrameLoader(prices)

## Run a Simple Backtest

In [ ]:
# Run backtest on 60/40 portfolio
result = rbt.backtest(
    portfolio={'SPY': 0.60, 'AGG': 0.40},
    loader=loader,
    start_date='2015-01-01',
    end_date='2023-12-31',
)

print(f"Backtest period: {len(result.daily_returns)} days")
print(f"Mean daily return: {result.daily_returns.mean():.6f}")
print(f"Daily volatility: {result.daily_returns.std():.6f}")

## RAS Analysis: The Components

RAS adjusts the empirical Sharpe ratio with two penalties:

$$\text{RAS-Adjusted Sharpe} = \text{Empirical Sharpe} - \text{Estimation Haircut} - \text{Complexity Haircut}$$

Let's break down each component:

In [ ]:
# Perform RAS analysis
ras = rbt.analyze_ras(
    returns=result.daily_returns,
    confidence=0.99,
    n_strategies=1,
)

# Display raw bounds (daily)
print("📊 RAS Components (Daily):")
print(f"  Empirical Sharpe: {ras.empirical_sharpe:.6f}")
print(f"  Estimation Haircut: {ras.estimation_haircut:.6f}")
print(f"  Complexity Haircut: {ras.complexity_haircut:.6f}")
print(f"  Total Haircut: {ras.total_haircut:.6f}")
print(f"  Adjusted Sharpe: {ras.adjusted_sharpe:.6f}")

print("\n📊 RAS Components (Annualized):")
print(f"  Empirical Sharpe: {ras.empirical_sharpe_annualized:.4f}")
print(f"  Estimation Haircut: {ras.estimation_haircut_annualized:.4f}")
print(f"  Complexity Haircut: {ras.complexity_haircut_annualized:.4f}")
print(f"  Total Haircut: {ras.total_haircut_annualized:.4f}")
print(f"  Adjusted Sharpe: {ras.adjusted_sharpe_annualized:.4f}")

### 1. Estimation Haircut

This accounts for **sampling error** from having finite data. Even a truly unprofitable strategy can appear profitable in a backtest due to random luck.

The estimation haircut is roughly:

$$\text{Estimation Haircut} \approx \frac{1}{\sqrt{T}}$$

where $T$ is the number of observations. More data = smaller haircut.

In [ ]:
# Visualize how estimation haircut decreases with more data
sample_sizes = np.arange(50, 2500, 50)
estimation_haircuts = []

for T in sample_sizes:
    # Use first T days
    returns_subset = result.daily_returns.iloc[:T]
    ras_subset = rbt.analyze_ras(returns_subset, confidence=0.99)
    estimation_haircuts.append(ras_subset.estimation_haircut_annualized)

plt.figure(figsize=(10, 6))
plt.plot(sample_sizes, estimation_haircuts, linewidth=2)
plt.xlabel('Sample Size (trading days)', fontsize=12)
plt.ylabel('Estimation Haircut (Annualized)', fontsize=12)
plt.title('Estimation Haircut vs Sample Size\n(Decreases as ~1/√T)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nWith {len(result.daily_returns)} days of data:")
print(f"  Estimation haircut ≈ {ras.estimation_haircut_annualized:.4f}")

### 2. Complexity Haircut

This accounts for **model complexity** and **data snooping**. If you tested many strategies and picked the best one, the winner likely benefited from luck.

The complexity haircut depends on:
- **Rademacher complexity**: How flexible your strategy space is
- **Number of strategies tested**: More tests = higher penalty

RAS estimates this using Monte Carlo simulation.

In [ ]:
# Show how complexity haircut increases with number of strategies tested
n_strategies_tested = [1, 2, 5, 10, 20, 50, 100]
complexity_haircuts = []

for N in n_strategies_tested:
    ras_multi = rbt.analyze_ras(result.daily_returns, confidence=0.99, n_strategies=N)
    complexity_haircuts.append(ras_multi.complexity_haircut_annualized)

plt.figure(figsize=(10, 6))
plt.plot(n_strategies_tested, complexity_haircuts, marker='o', linewidth=2, markersize=8)
plt.xlabel('Number of Strategies Tested', fontsize=12)
plt.ylabel('Complexity Haircut (Annualized)', fontsize=12)
plt.title('Complexity Haircut vs Number of Strategies\n(Multiple Testing Penalty)', 
         fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.xscale('log')
plt.tight_layout()
plt.show()

print(f"\nComplexity penalty for testing different numbers of strategies:")
for N, haircut in zip(n_strategies_tested, complexity_haircuts):
    print(f"  {N:3d} strategies: +{haircut:.4f} haircut")

## Statistical Significance Test

RAS provides a formal test:

**Null Hypothesis**: The strategy has zero or negative Sharpe ratio

**Decision Rule**: 
- If RAS-adjusted Sharpe > 0, we **reject the null** at the specified confidence level
- The strategy is **statistically positive**

In [ ]:
print("\n🔬 Statistical Test Results:")
print(f"  Confidence Level: {ras.confidence_level:.0f}%")
print(f"  Empirical Sharpe: {ras.empirical_sharpe_annualized:.4f}")
print(f"  RAS-Adjusted Sharpe: {ras.adjusted_sharpe_annualized:.4f}")
print(f"  Statistically Positive: {'YES ✅' if ras.is_statistically_positive else 'NO ❌'}")

print(f"\n{ras.interpretation}")

## Visualize the Haircut Decomposition

In [ ]:
# Create waterfall chart showing haircut breakdown
components = [
    ('Empirical\nSharpe', ras.empirical_sharpe_annualized),
    ('Estimation\nHaircut', -ras.estimation_haircut_annualized),
    ('Complexity\nHaircut', -ras.complexity_haircut_annualized),
    ('RAS-Adjusted\nSharpe', ras.adjusted_sharpe_annualized),
]

fig, ax = plt.subplots(figsize=(12, 7))

# Bar positions
x = np.arange(len(components))
values = [c[1] for c in components]
labels = [c[0] for c in components]

# Colors
colors = ['#2ecc71', '#e74c3c', '#e74c3c', '#3498db']

bars = ax.bar(x, values, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)

# Add value labels on bars
for i, (bar, val) in enumerate(zip(bars, values)):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{val:.4f}',
            ha='center', va='bottom' if val > 0 else 'top',
            fontsize=11, fontweight='bold')

ax.set_xticks(x)
ax.set_xticklabels(labels, fontsize=11)
ax.set_ylabel('Sharpe Ratio (Annualized)', fontsize=12)
ax.set_title('RAS Haircut Decomposition', fontsize=14, fontweight='bold')
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## Effect of Confidence Level

Higher confidence levels demand more conservative bounds:

In [ ]:
confidence_levels = [0.90, 0.95, 0.99, 0.999]
adjusted_sharpes = []

for conf in confidence_levels:
    ras_conf = rbt.analyze_ras(result.daily_returns, confidence=conf)
    adjusted_sharpes.append(ras_conf.adjusted_sharpe_annualized)

plt.figure(figsize=(10, 6))
plt.plot([c * 100 for c in confidence_levels], adjusted_sharpes, 
         marker='o', linewidth=2, markersize=10)
plt.axhline(y=ras.empirical_sharpe_annualized, color='green', 
           linestyle='--', label='Empirical Sharpe', linewidth=2)
plt.xlabel('Confidence Level (%)', fontsize=12)
plt.ylabel('RAS-Adjusted Sharpe (Annualized)', fontsize=12)
plt.title('Effect of Confidence Level on RAS Adjustment', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nRAS-Adjusted Sharpe at different confidence levels:")
for conf, adj_sharpe in zip(confidence_levels, adjusted_sharpes):
    print(f"  {conf*100:5.1f}% confidence: {adj_sharpe:.4f}")

## Practical Implications

### 1. More Data = Better
The estimation haircut decreases with sample size. Longer backtests provide tighter bounds.

### 2. Testing Multiple Strategies is Costly
Each additional strategy tested increases the complexity haircut. This is the **multiple testing problem**.

### 3. Simple Strategies Preferred
Complex strategies with many parameters have higher Rademacher complexity, leading to larger haircuts.

### 4. Out-of-Sample is Still Important
RAS provides a statistical bound, but true out-of-sample testing remains the gold standard.

### 5. Use for Go/No-Go Decisions
If RAS-adjusted Sharpe ≤ 0, the strategy is **not statistically distinguishable from zero**. Don't trade it!

## Summary

The Rademacher Anti-Serum (RAS) methodology provides:

1. **Rigorous statistical bounds** on the Sharpe ratio
2. **Correction for data snooping** and multiple testing
3. **Penalty for model complexity** and overfitting
4. **Clear decision rule** for strategy viability

Key formula:

$$\text{RAS-Adjusted Sharpe} = \text{Empirical Sharpe} - \frac{\text{Estimation}}{\sqrt{T}} - \text{Complexity}(N, \mathcal{F})$$

Where:
- $T$ = sample size
- $N$ = number of strategies tested
- $\mathcal{F}$ = strategy complexity

**Bottom Line**: If your RAS-adjusted Sharpe ratio is positive, you have **statistically significant evidence** of outperformance, accounting for all the ways backtest results can be misleading.

## References

- Bailey & López de Prado (2012): "The Sharpe Ratio Efficient Frontier"
- Bailey, Borwein, López de Prado, & Zhu (2014): "Pseudomathematics and Financial Charlatanism: The Effects of Backtest Overfitting on Out-of-Sample Performance"
- López de Prado (2018): *Advances in Financial Machine Learning*, Chapter 8